In [4]:
import wbdata  # For World Bank data
import pandas as pd
import datetime as dt
from fredapi import Fred
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')



In [7]:
file_path='C:/Users/Administrator/Documents/kifiya/Week_10/data/preproccessed_brent_oil_prices.csv'
df = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)
df = df.sort_values(by='Date')

display(df.head())


,Date,Price,Returns,Log_Returns,Volatility,Momentum,Rolling_Corr
0,1987-06-19,19.05,0.000000,0.000000,0.004308,0.37,NaN
1,1987-06-22,19.10,0.002625,0.002621,0.003544,0.32,NaN
2,1987-06-23,18.90,-0.010471,-0.010526,0.004313,0.00,NaN
3,1987-06-24,18.75,-0.007937,-0.007968,0.004698,-0.28,NaN
4,1987-06-25,18.70,-0.002667,-0.002670,0.004736,-0.35,NaN


In [11]:
# Set up FRED API
fred_api_key = 'cccb814f569f57a4104878fa9cb9d389'
fred = Fred(api_key=fred_api_key)

# Define the data period to match the Brent oil prices dataset
start_date = dt.datetime(1987, 5, 20)
end_date = dt.datetime(2022, 9, 30)

countries = ['USA', 'CHN', 'EMU']  # USA: United States, CHN: China, EMU: Euro Area
indicators = {
    'NY.GDP.MKTP.CD': 'GDP',
    'FP.CPI.TOTL': 'Inflation',
    'SL.UEM.TOTL.ZS': 'Unemployment'
}


In [ ]:
# Fetch data from World Bank API using date strings# Fetch data from World Bank API using datetime filtering directly
wb_data = wbdata.get_dataframe(indicators, country=countries, convert_date=True)

# Filter the data to the desired date range
wb_data = wb_data[(wb_data.index >= start_date) & (wb_data.index <= end_date)]

# Reshape and clean data
wb_data = wb_data.unstack().reset_index()
wb_data.columns = ['Country', 'Indicator', 'Date', 'Value']

In [13]:
# Fetch GDP data for major economies (e.g., United States, China, Euro Area)
countries = ['USA', 'CHN', 'EMU']  # USA: United States, CHN: China, EMU: Euro Area

gdp_data = wbdata.get_dataframe(indicator='NY.GDP.MKTP.CD', country=countries, data_date=(start_date, end_date))
gdp_data = gdp_data.unstack().reset_index()
gdp_data.columns = ['Country', 'Date', 'GDP']
gdp_data.set_index('Date', inplace=True)
gdp_data.sort_index(inplace=True)


TypeError: missing a required argument: 'indicators'

In [ ]:
# Fetch Inflation data (consumer prices) for major economies
inflation_data = wbdata.get_dataframe(indicator='FP.CPI.TOTL', country=countries, data_date=(start_date, end_date))
inflation_data = inflation_data.unstack().reset_index()
inflation_data.columns = ['Country', 'Date', 'Inflation']
inflation_data.set_index('Date', inplace=True)
inflation_data.sort_index(inplace=True)


In [ ]:
# Fetch Unemployment data for major economies
unemployment_data = wbdata.get_dataframe(indicator='SL.UEM.TOTL.ZS', country=countries, data_date=(start_date, end_date))
unemployment_data = unemployment_data.unstack().reset_index()
unemployment_data.columns = ['Country', 'Date', 'Unemployment']
unemployment_data.set_index('Date', inplace=True)
unemployment_data.sort_index(inplace=True)


In [ ]:
# Fetch USD/EUR exchange rate data
exchange_rate_data = fred.get_series('DEXUSEU', start_date, end_date)
exchange_rate_data = exchange_rate_data.to_frame(name='Exchange_Rate')
exchange_rate_data.index.name = 'Date'
exchange_rate_data.sort_index(inplace=True)


In [ ]:
# Assuming 'data' is the original Brent oil prices DataFrame from Task 1
# Ensure all datasets are at the same frequency (monthly/quarterly) if required

# Merge GDP, Inflation, Unemployment, and Exchange Rate into the main dataset
data = data.merge(gdp_data, how='left', left_index=True, right_index=True)
data = data.merge(inflation_data, how='left', left_index=True, right_index=True)
data = data.merge(unemployment_data, how='left', left_index=True, right_index=True)
data = data.merge(exchange_rate_data, how='left', left_index=True, right_index=True)

# Display merged data
print(data.head())


In [ ]:
# Plot the Brent Oil Price and Economic Indicators
fig, axes = plt.subplots(3, 2, figsize=(14, 10), sharex=True)
fig.suptitle('Brent Oil Price and Economic Indicators')

axes[0, 0].plot(data['Price'], label='Brent Oil Price', color='blue')
axes[0, 0].set_ylabel('Brent Oil Price (USD)')
axes[0, 0].legend()

axes[0, 1].plot(data['GDP'], label='GDP', color='green')
axes[0, 1].set_ylabel('GDP (USD)')
axes[0, 1].legend()

axes[1, 0].plot(data['Inflation'], label='Inflation', color='purple')
axes[1, 0].set_ylabel('Inflation (%)')
axes[1, 0].legend()

axes[1, 1].plot(data['Unemployment'], label='Unemployment', color='red')
axes[1, 1].set_ylabel('Unemployment (%)')
axes[1, 1].legend()

axes[2, 0].plot(data['Exchange_Rate'], label='Exchange Rate (USD/EUR)', color='orange')
axes[2, 0].set_ylabel('Exchange Rate')
axes[2, 0].legend()

plt.show()

# Correlation matrix
correlation_matrix = data[['Price', 'GDP', 'Inflation', 'Unemployment', 'Exchange_Rate']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm")
plt.title("Correlation Matrix between Brent Oil Price and Economic Indicators")
plt.show()
